In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.types as t
import pyspark.sql.functions as f
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd

SEED_VALUE = 42

In [2]:
spark = (SparkSession.builder.config("spark.driver.memory","5g").config("spark.driver.maxResultSize", "5g").getOrCreate())
spark

In [ ]:
%%time
from pyspark.sql.types import DoubleType


X_train = spark.read.csv("./Dataset/train.csv", header=True, inferSchema=True)
X_test = spark.read.csv("./Dataset/example_test.csv", header=True, inferSchema=True)

# Convert VWAP Feature to double
X_train = X_train.withColumn("VWAP", X_train.VWAP.cast(DoubleType()))

X_train.printSchema()

In [ ]:
X_train.head(1)

In [ ]:
%%time
# Check null values if any

X_train.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in X_train.columns]).show()

In [ ]:
# Drop missing values
X_train = X_train.na.drop()

In [ ]:
%%time
# Check null values if any

X_train.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in X_train.columns]).show()

In [ ]:
col_features = ['Count', "Open", "High", "Low", "Close", "Volume"]

In [ ]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=col_features, outputCol="features")
output = vecAssembler.transform(X_train).select('features', 'Target')

output.select('features').show()

In [ ]:
# Base Model (Without Time series considerations)
from pyspark.ml.regression import LinearRegression
train_data, test_data = output.randomSplit([0.75, 0.25]) # Randomly split the data into training and testing

lr = LinearRegression(featuresCol="features", labelCol = "Target")

lrModel = lr.fit(train_data) # Fit the model

In [ ]:
lrModel.coefficients

In [ ]:
lrModel.intercept

In [15]:
%%time
# Predict/Evaluate on test data

pred_results = lrModel.evaluate(test_data)
pred_results.predictions.show()

/home/sameep/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+--------------------+--------------------+
|            features|              Target|          prediction|
+--------------------+--------------------+--------------------+
|[1.0,0.0251,0.025...|7.785829789781484E-4|-3.23497948120739...|
|[1.0,0.030485,0.0...|-0.01314475873544...|-3.51733445792442...|
|[1.0,0.03055,0.03...|0.007245901639344243|-3.50072249715613...|
|[1.0,0.0309,0.030...|0.013009708737864223|-3.42106101244415...|
|[1.0,0.0310170000...|0.001958224543081144|-3.20289515509599...|
|[1.0,0.03102,0.03...|-0.02023690410870...|-3.46525996110049...|
|[1.0,0.03184,0.03...|0.026107694238734824|-3.50332191630705...|
|[1.0,0.03193,0.03...|-0.03343749999999979|-3.40981882241790...|
|[1.0,0.032,0.032,...|-0.03998124413879...|-3.50583064507680...|
|[1.0,0.032234,0.0...|0.010310555159922075|-3.47381560174487...|
|[1.0,0.0322470000...|-0.01086517865458...|-3.22566142574302...|
|[1.0,0.0323100000...|-0.00675925925925...|-3.49029660570717...|
|[1.0,0.03258,0.03...|-6.

In [16]:
%%time

X_train = pd.read_csv("./Dataset/train.csv")
X_train.head()

CPU times: user 24.5 s, sys: 3.4 s, total: 27.9 s
Wall time: 28.2 s


,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [17]:
X_train.dropna(inplace=True)
X_train.describe()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
count,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07,2.348646e+07
mean,1.577541e+09,6.224257e+00,2.954028e+02,1.476051e+03,1.479876e+03,1.472882e+03,1.476050e+03,2.947317e+05,NaN,7.121863e-06
std,3.333367e+07,4.099476e+00,8.796244e+02,6.119951e+03,6.129966e+03,6.110473e+03,6.119956e+03,2.471421e+06,NaN,5.679042e-03
min,1.514765e+09,0.000000e+00,1.000000e+00,1.170400e-03,1.195000e-03,2.000000e-04,1.171400e-03,-3.662812e-01,-inf,-5.093509e-01
25%,1.549306e+09,3.000000e+00,2.200000e+01,2.808670e-01,2.819000e-01,2.800000e-01,2.808736e-01,1.493896e+02,2.808644e-01,-1.694353e-03
50%,1.578971e+09,6.000000e+00,6.900000e+01,1.486842e+01,1.489000e+01,1.484400e+01,1.486866e+01,1.332360e+03,1.486720e+01,-4.289820e-05
75%,1.606979e+09,9.000000e+00,2.310000e+02,2.349650e+02,2.353800e+02,2.345200e+02,2.349686e+02,2.911512e+04,2.349653e+02,1.601519e-03
max,1.632181e+09,1.300000e+01,1.650160e+05,6.480594e+04,6.490000e+04,6.467053e+04,6.480854e+04,7.597554e+08,inf,9.641699e-01


In [18]:
y_train = X_train['Target']
X_train = X_train.drop(["Target", "VWAP"], axis=1)

In [19]:
from sklearn.model_selection import train_test_split
X_train_1, X_test, y_train_1, y_test = train_test_split(X_train, y_train, random_state=SEED_VALUE, train_size=0.80)

In [1]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

fitted_lr_model = lr_model.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [ ]:
preds = fitted_lr_model.predict(X_test)
preds

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, preds)
mae